In [1]:
import os
import numpy as np

from jsonschema import validate, ValidationError, Draft7Validator

from astropy.io import fits
from pyaraucaria.ffs import FFS



In [2]:
# konfiguracja schema

def normalize_header(header):
    out = dict(header)

    for k, v in out.items():
        if isinstance(v, str):
            out[k] = v.strip()
        if v == 'T':
            out[k] = True
        if v == 'F':
            out[k] = False

    return out


SCIENCE_SCHEMA = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "required": [
        "IMAGETYP",
        "MIRR-COV",
        "DOME-SHT",
        "ALT_TEL",
        "EXPTIME",
        "T-CAM",
        "T-CAMSET",
        "TRACKING",
        "RHUM-WS",
        "T-WS",
        "WIND-AVG"
    ],
    "properties": {
        "IMAGETYP": {
            "type": "string",
            "enum": ["science"]
        },

        "MIRR-COV": {
            "type": "integer",
            "enum": [3]  # open
        },

        "DOME-SHT": {
            "type": "integer",
            "enum": [0, 2]  # open / opening
        },

        "ALT_TEL": {
            "type": "number",
            "minimum": 34
        },

        "EXPTIME": {
            "type": "number",
            "Minimum": 0.5
        },

        "T-CAM": {
            "type": "number",
            "maximum": -58
        },

        "T-CAMSET": {
            "type": "number",
            "enum": [-60.0]
        },

        "READ-MOD": {
            "type": "integer",
            "const": 2
        },

        "GAIN-MOD": {
            "type": "integer",
            "const": 2
        },

        "TRACKING": {
            "type": "boolean",
            "const": True
        },

        "FAN-MIRR": {
            "type": "boolean",
            "const": False
        },

        "FAN-DOME": {
            "type": "boolean",
            "const": False
        },

        "RHUM-WS": {
            "type": "number",
            "maximum": 75
        },

        "T-WS": {
            "type": "number",
            "minimum": 0
        },

        "WIND-AVG": {
            "type": "number",
            "maximum": 15
        }
    },

    "additionalProperties": True
}




In [9]:
# General Filter

def ffs_qc(f_name,hdr,ffs_stats,saturation=40000):
    objects_2_ignore = ["NGC2362"]

    if hdr["OBJECT"] not in objects_2_ignore:

        f = f_name

        sc = f + " "
        dqc = False
        txt = ""

        # header
        hdr = normalize_header(hdr)

        validator = Draft7Validator(SCIENCE_SCHEMA)
        for err in validator.iter_errors(hdr):
            dqc = True
            sc = sc + "❌"
            txt = txt + f'❌ {list(err.path)} : {err.message} \n'

        # LINIE

        c = ffs_stats["lines"]["val"]

        if len(c) > 0:
            c = c[0]
            if c > 500:
                dqc = True
                sc = sc + "*"
                txt = txt + f'* LINE detected {c} \n'

        # GRADIENT
        c = ffs_stats["bkg_frame_gradient"]
        if c > 100:
            dqc = True
            sc = sc + "*"
            txt = txt + f'* FRAME GRADIENT detected {c} \n'
        c = ffs_stats["bkg_max_amplitude"]
        if c > 100:
            dqc = True
            sc = sc + "*"
            txt = txt + f'* LARGE SKY BACKGROUND CHANGES detected {c} \n'

        # czy sa gwiazdy
        stars = ffs_stats["stars"]
        mk = stars["max_adu"] < saturation

        c = len(np.array(stars["x"])[mk])
        if c < 5:
            dqc = True
            sc = sc + "#"
            txt = txt + f'# NO STARS detected {c} \n'
        else:
            # FWHM
            if "fwhm_x" in stars.keys():

                if len(np.array(stars["fwhm_x"])[mk]) < 10:
                    dqc = True
                    sc = sc + "#"
                    txt = txt + f'# NO FWHM measured {c} \n'
                else:
                    fx = np.nanmedian(np.array(stars["fwhm_x"])[mk])
                    fy = np.nanmedian(np.array(stars["fwhm_y"])[mk])
                    if np.isnan(fx) or np.isnan(fy):
                        dqc = True
                        sc = sc + "*"
                        txt = txt + f'* FWHM: {fx} {fy} \n'
                    else:
                        fwhm = fx + fy / 2
                        if fwhm > 14:  # TUTEJ
                            dqc = True
                            sc = sc + "*"
                            txt = txt + f'* FWHM: {fwhm:.2f} \n'
            else:
                dqc = True
                sc = sc + "#"
                txt = txt + f'# fwhm ERROR {c} \n'

            # ellipcity
            if "ellipticity" in stars.keys():
                if len(np.array(stars["ellipticity"])[mk]) < 10:
                    dqc = True
                    sc = sc + "#"
                    txt = txt + f'# no EL measured {c} \n'
                else:
                    el = np.nanmedian(np.array(stars["ellipticity"])[mk])
                    if np.isnan(el):
                        dqc = True
                        sc = sc + "*"
                        txt = txt + f'* ellipticity: {el:.2f} \n'
                    else:
                        if el > 0.3:  # TUTEJ
                            dqc = True
                            sc = sc + "*"
                            txt = txt + f'* ellipticity: {el:.2f} \n'
                        # if el > 0.15:
                        #     dqc = True
                        #     sc = sc + "o"
                        #     txt = txt + f'* ellipticity: {el:2.f} \n'
            else:
                dqc = True
                sc = sc + "#"
                txt = txt + f'# fwhm ERROR {c} \n'

        if dqc:
            print(f'{sc} {hdr["OBJECT"]}')
            print(txt)


In [10]:
fits_path = "~/mglab/data/fits/zb08c_1052_65023.fits"

hdul = fits.open(fits_path)
data = hdul[0].data
hdr = hdul[0].header
hdul.close()

ffs = FFS(data)
ffs.saturation = 50000
ffs.mk_stats()
ffs.sky_gradient(n_segments=10)
maska = data > np.median(data) + 3 * ffs.q_sigma
ffs.hough_transform(maska)
ffs.find_stars(threshold=10,fwhm=3)
ffs.star_info(box=15,N_stars=20)

print(ffs.mean)

384.83482575416565


In [11]:
ffs_qc(fits_path,hdr,ffs.stats)

~/mglab/data/fits/zb08c_1052_65023.fits * em01_115120_020_ML00018
* LINE detected 255.0 

